In [94]:
## import relevant packages: numpy, pandas, networkx
import numpy as np
import pandas as pd
import networkx as nx
import time

# Part 1: Initiate the Patent Graph

#### Note: I'm using Networkx package of Python to do this replication. Networkx is suitable for this situation because the citation tree proposed by Belenzon can be perfectly achieved by a DiGraph (directed graph) in Networkx, where the direction is from cited patent to citing patent. We are also allowed to set attributes for each node (here patent) in Networkx, so we can easily include its granted year, assignee's code as well as the label (internal, external, self) into the node. 

In [ ]:
# read data from dta files
# You should change the directory of these data to your local directory
compu = pd.read_stata('compustat(2).dta') # compustat data (link gvkeys to 9-digit CUSIP numbers)
cusip = pd.read_stata('CUSIP(2).dta') # cusip data (link acquiree numbers to 6-digit CUSIP numbers)
pairwise = pd.read_stata('pairwise with info.dta') # patent pairwise data (cited patent numbers and citing patent numbers)
P = pd.read_stata('Patent data, including constructed variables(1963_1999).dta') # patent data (link patent numbers to acquiree numbers)

In [3]:
P = P[~P['assignee'].isnull()] # remove the patents that do not have an assignee number
P['assignee'] = P['assignee'].map(int)  # map the granted year of patents to integer
pairwise = pairwise[~pairwise['assignee'].isnull()] # remove the pairwise data that do not have assignee numbers for the cited patents
pairwise = pairwise[~pairwise['assignee1'].isnull()] # remove the pairwise data that do not have assignee numbers for the citing patents
pairwise['assignee'] = pairwise['assignee'].map(int) # map the assignee numbers of cited patents to integer
pairwise['assignee1'] = pairwise['assignee1'].map(int) # map the assignee numbers of citing patents to integer
pairwise['gyear'] = pairwise['gyear'].map(int) # map the granted year of cited patents to integer
pairwise['gyear1'] = pairwise['gyear1'].map(int) # map the granted year of citing patents to integer

In [4]:
edges_list = pairwise[['cited','citing1']] # create a edge list from pairwise data, using the 'citing' and 'cited' columns
# In each edge, cited patent is predecessor, citing patent is successor

In [5]:
edges_list = [tuple(i) for i in np.asarray(edges_list)] # change the edge list to the correct format for networkx

In [6]:
G = nx.DiGraph() # initiate a directed graph

In [7]:
G.add_edges_from(edges_list) # add all the edges from the edge list to the directed graph
# The direction is from cited to citing patent

In [8]:
ind = list(G.in_degree) # compute the in-degree for all the nodes (patents) in directed graph

# Part 2: Patent Graph Characteristics

#### Note: I want to see how many patents are 'original patents'. In P270 of Belenzon's paper, it says that 'the sample includes 201943 original patents that are granted between 1969 and 1990'. However, I find that there are 322319 original patents if we do not exclude those patents whose assignees are not Compustat firms. If we do consider this restriction, there will be 156851 original patents. In either case the figure cannot match Belenzon's figure. 

In [9]:
original = [i[0] for i in ind if i[1] == 0] # for all nodes (patents) in the graph, we select those who have in-degree equals to zero
# These nodes are 'original' patents, because they have not been cited by any other patents

In [10]:
len(original) # We find that 383503 patents are 'original' patents

383503

In [11]:
N = list(G.nodes()) # Store all the nodes (patents) in the graph to a list called N

In [12]:
# Create a dictionary where the keys are patents and the values are granted years and assignee numbers
gyear = P['gyear'].to_list()
patent = P['patent'].to_list()
assignee = P['assignee'].to_list()
attr_list = [{'year': gyear[i], 'assignee': assignee[i]} for i in range(len(patent))]
attr = [(patent[i],attr_list[i]) for i in range(len(patent))]
attr = dict(attr)

In [13]:
# Add those attributes into the graph
nx.set_node_attributes(G, attr)

In [14]:
# Only select patents that are granted between 1969 and 1990 from all the original patents
# This is according to P269 and P270 of Belenzon's paper
original_1969_1990 = [i for i in original if G.node[i]['year'] > 1969 and G.node[i]['year'] < 1990]

In [15]:
len(original_1969_1990) # We find that 322319 patents are 'original' patents that are granted between 1969 and 1990
# This is very different from Belenzon's finding (he finds 201943 such patents)

322319

In [16]:
# Belenzon also mentioned that he only included citations by patents that are held by Compustat firms
# This time we only select patents that are held by Compustat firm
original_1969_1990_compu = [i for i in original_1969_1990 if G.node[i]['assignee'] in cusip['assignee'].tolist()]

In [18]:
len(original_1969_1990_compu) # I find that in this case there are 156851 patents, still not in align with Belenzon

156851

#### Note: On page 271 Belenzon examines the percentage of each patent generation. I try to replicate this as well, but get a very different result. Here I do the following restriction: a patent can only be classified as a unique generation which is the smallest among all the possibilities. For example, if a patent is classified as both the 2nd generation and the 3rd generation according to the graph, it will be only classified as 2nd generation eventually.

In [83]:
original_1969_1990_offsprings1 = [list(G.successors(i)) for i in original_1969_1990_compu]
original_1969_1990_offsprings1 = [i for sublist in original_1969_1990_offsprings1 for i in sublist]
original_1969_1990_offsprings1 = list(set(original_1969_1990_offsprings1))
original_1969_1990_compu_offsprings1 = [i for i in original_1969_1990_offsprings1 if G.node[i]['assignee'] in cusip['assignee'].tolist()]
original_1969_1990_compu_offsprings1 = list(set(original_1969_1990_compus_offsprings1).difference(set(original_1969_1990_compu)))
len(original_1969_1990_compu_offsprings1)  # I find 272519 first-generation offspring patents

272519

In [85]:
original_1969_1990_offsprings2 = [list(G.successors(i)) for i in original_1969_1990_compu_offsprings1]
original_1969_1990_offsprings2 = [i for sublist in original_1969_1990_offsprings2 for i in sublist]
original_1969_1990_offsprings2 = list(set(original_1969_1990_offsprings2))
original_1969_1990_compu_offsprings2 = [i for i in original_1969_1990_offsprings2 if G.node[i]['assignee'] in cusip['assignee'].tolist()]
original_1969_1990_compu_offsprings2 = list(set(original_1969_1990_compus_offsprings2).difference(set(original_1969_1990_compu_offsprings1)).difference(set(original_1969_1990_compu)))
len(original_1969_1990_compu_offsprings2)  # I find 239543 second-generation offspring patents

239543

In [87]:
original_1969_1990_offsprings3 = [list(G.successors(i)) for i in original_1969_1990_compu_offsprings2]
original_1969_1990_offsprings3 = [i for sublist in original_1969_1990_offsprings3 for i in sublist]
original_1969_1990_offsprings3 = list(set(original_1969_1990_offsprings3))
original_1969_1990_compu_offsprings3 = [i for i in original_1969_1990_offsprings3 if G.node[i]['assignee'] in cusip['assignee'].tolist()]
original_1969_1990_compu_offsprings3 = list(set(original_1969_1990_compu_offsprings3).difference(set(original_1969_1990_compu_offsprings2)).difference(set(original_1969_1990_compu_offsprings1)).difference(set(original_1969_1990_compu)))
len(original_1969_1990_compu_offsprings3)  # I find 38894 third-generation offspring patents


38894

In [89]:
original_1969_1990_offsprings4 = [list(G.successors(i)) for i in original_1969_1990_compu_offsprings3]
original_1969_1990_offsprings4 = [i for sublist in original_1969_1990_offsprings4 for i in sublist]
original_1969_1990_offsprings4 = list(set(original_1969_1990_offsprings4))
original_1969_1990_compu_offsprings4 = [i for i in original_1969_1990_offsprings4 if G.node[i]['assignee'] in cusip['assignee'].tolist()]
original_1969_1990_compu_offsprings4 = list(set(original_1969_1990_compu_offsprings4).difference(set(original_1969_1990_compu_offsprings3)).difference(set(original_1969_1990_compu_offsprings2)).difference(set(original_1969_1990_compu_offsprings1)).difference(set(original_1969_1990_compu)))
len(original_1969_1990_compu_offsprings4) # I find 3388 fourth-generation offspring patents


3388

In [90]:
original_1969_1990_offsprings5 = [list(G.successors(i)) for i in original_1969_1990_compu_offsprings4]
original_1969_1990_offsprings5 = [i for sublist in original_1969_1990_offsprings5 for i in sublist]
original_1969_1990_offsprings5 = list(set(original_1969_1990_offsprings5))
original_1969_1990_compu_offsprings5 = [i for i in original_1969_1990_offsprings5 if G.node[i]['assignee'] in cusip['assignee'].tolist()]
original_1969_1990_compu_offsprings5 = list(set(original_1969_1990_compu_offsprings5).difference(set(original_1969_1990_compu_offsprings4)).difference(set(original_1969_1990_compu_offsprings3)).difference(set(original_1969_1990_compu_offsprings2)).difference(set(original_1969_1990_compu_offsprings1)).difference(set(original_1969_1990_compu)))
len(original_1969_1990_compu_offsprings5) 
# I find 296 fifth-generation offspring patents


296

In [40]:
N_compu = [i for i in N if G.node[i]['assignee'] in cusip['assignee'].tolist()] 
## total patents assigned by Compustat firms

In [170]:
N_offsprings = len(N_compu) - len(original_1969_1990_compu) ## number of offspring patents 

In [177]:
print('{:f}'.format(len(original_1969_1990_compu_offsprings1)*100/N_offsprings) + '% data are first generation')
print('{:f}'.format(len(original_1969_1990_compu_offsprings2)*100/N_offsprings) + '% data are second generation')
print('{:f}'.format(len(original_1969_1990_compu_offsprings3)*100/N_offsprings) + '% data are third generation')
print('{:f}'.format(len(original_1969_1990_compu_offsprings4)*100/N_offsprings) + '% data are fourth generation')
print('{:f}'.format(len(original_1969_1990_compu_offsprings5)*100/N_offsprings) + '% data are fifth generation')

42.728440% data are first generation
37.558111% data are second generation
6.098217% data are third generation
0.531207% data are fourth generation
0.046410% data are fifth generation


# Part 3: Table 2 Generation

#### Here I use a home-made algorithm to compute the share of internal/external/self citations. There is a detailed description about the algorithm in a seperate pdf file. First of all, I'm going to replicate the 'Internal Citation Sequence' in Fig.1 on page 270. The replication is successful.

In [361]:
# The algorithm.
# See the attached pdf file
# Replicate the 'Internal Citation Sequence' in Fig.1 on page 270

T = nx.algorithms.bfs_tree(G,3462742)
TN = list(T.nodes())
TN_list = [(i,attr[i]) for i in TN]
TN_dict = dict(TN_list)
nx.set_node_attributes(T,TN_dict)

for n in TN:
    T.node[n]['label'] = 'none'

TN.remove(3462742)
for n in TN:
    if T.node[n]['assignee'] == T.node[3462742]['assignee']:
        T.node[n]['label'] = 'self'
    else:
        T.node[n]['label'] = 'external'

self_nodes = [i[0] for i in list(T.nodes(data = 'label')) if i[1] == 'self']

pred_list = self_nodes
while len(pred_list) > 0:
    pred_list = [list(T.predecessors(i)) for i in pred_list]
    pred_list = [i for sublist in pred_list for i in sublist]
    for p in pred_list:
        if T.node[p]['label'] != 'self' and T.node[p]['label'] != 'none':
            T.node[p]['label'] = 'internal'

In [363]:
T.node[3983538]['label']  # label of patent 3983538, this is internal according to the paper

'internal'

In [364]:
T.node[4084230]['label']  # label of patent 3983538, this is internal according to the paper

'internal'

In [365]:
T.node[4276609]['label']  # label of patent 3983538, this is internal according to the paper

'internal'

In [366]:
T.node[4454541]['label']  # label of patent 3983538, this is self according to the paper

'self'

In [372]:
T.node[4446514]['label']

'external'

In [179]:
# Patents that originate between 1975 and 1999
original_1975_1999 = [i for i in original if G.node[i]['year'] > 1975 and G.node[i]['year'] < 1999]

In [180]:
# Patents that originate between 1975 and 1999 and are assigned by Compustat firms
original_1975_1999_compu = [i for i in original_1975_1999 if G.node[i]['assignee'] in cusip['assignee'].tolist()]

In [414]:
# Create five empty lists to hold the results from the algorithm
share_internal_list = []
share_self_list = []
share_external_list = []
total_citation_list = []
direct_citation_list= []

In [415]:
# The algorithm
# See the attached pdf file
# This take several hours to run
u = 0
for k in original_1975_1999_compu:
    T = nx.algorithms.bfs_tree(G,k)
    TN = list(T.nodes())
    TN_list = [(i,attr[i]) for i in TN]
    TN_dict = dict(TN_list)
    nx.set_node_attributes(T,TN_dict)
    
    for n in TN:
        T.node[n]['label'] = 'none'
    
    TN.remove(k)
    for n in TN:
        if T.node[n]['assignee'] == T.node[k]['assignee']:
            T.node[n]['label'] = 'self'
        else:
            T.node[n]['label'] = 'external'
            
    self_nodes = [i[0] for i in list(T.nodes(data = 'label')) if i[1] == 'self']
    
    pred_list = self_nodes
    while len(pred_list) > 0:
        pred_list = [list(T.predecessors(i)) for i in pred_list]
        pred_list = [i for sublist in pred_list for i in sublist]
        for p in pred_list:
            if T.node[p]['label'] != 'self' and T.node[p]['label'] != 'none':
                T.node[p]['label'] = 'internal'
                
    label_list = [i[1] for i in list(T.nodes(data = 'label'))]
    share_internal = len([i for i in label_list if i == 'internal'])/(len(label_list) - 1)
    share_self = len([i for i in label_list if i == 'self'])/(len(label_list) - 1)
    share_external = len([i for i in label_list if i == 'external'])/(len(label_list) - 1)
    total_citation = len(TN)
    direct_citation = len(list(T.successors(k)))
    
    share_internal_list.append(share_internal)
    share_self_list.append(share_self)
    share_external_list.append(share_external)
    total_citation_list.append(total_citation)
    direct_citation_list.append(direct_citation)
    
    u += 1
    print('number of patents finished processing: ' + str(u) + ', total: ' + str(len(original_1975_1999_compu)))

number of patents finished processing: 1, total: 29216
number of patents finished processing: 2, total: 29216
number of patents finished processing: 3, total: 29216
number of patents finished processing: 4, total: 29216
number of patents finished processing: 5, total: 29216
number of patents finished processing: 6, total: 29216
number of patents finished processing: 7, total: 29216
number of patents finished processing: 8, total: 29216
number of patents finished processing: 9, total: 29216
number of patents finished processing: 10, total: 29216
number of patents finished processing: 11, total: 29216
number of patents finished processing: 12, total: 29216
number of patents finished processing: 13, total: 29216
number of patents finished processing: 14, total: 29216
number of patents finished processing: 15, total: 29216
number of patents finished processing: 16, total: 29216
number of patents finished processing: 17, total: 29216
number of patents finished processing: 18, total: 29216
n

number of patents finished processing: 148, total: 29216
number of patents finished processing: 149, total: 29216
number of patents finished processing: 150, total: 29216
number of patents finished processing: 151, total: 29216
number of patents finished processing: 152, total: 29216
number of patents finished processing: 153, total: 29216
number of patents finished processing: 154, total: 29216
number of patents finished processing: 155, total: 29216
number of patents finished processing: 156, total: 29216
number of patents finished processing: 157, total: 29216
number of patents finished processing: 158, total: 29216
number of patents finished processing: 159, total: 29216
number of patents finished processing: 160, total: 29216
number of patents finished processing: 161, total: 29216
number of patents finished processing: 162, total: 29216
number of patents finished processing: 163, total: 29216
number of patents finished processing: 164, total: 29216
number of patents finished proc

number of patents finished processing: 292, total: 29216
number of patents finished processing: 293, total: 29216
number of patents finished processing: 294, total: 29216
number of patents finished processing: 295, total: 29216
number of patents finished processing: 296, total: 29216
number of patents finished processing: 297, total: 29216
number of patents finished processing: 298, total: 29216
number of patents finished processing: 299, total: 29216
number of patents finished processing: 300, total: 29216
number of patents finished processing: 301, total: 29216
number of patents finished processing: 302, total: 29216
number of patents finished processing: 303, total: 29216
number of patents finished processing: 304, total: 29216
number of patents finished processing: 305, total: 29216
number of patents finished processing: 306, total: 29216
number of patents finished processing: 307, total: 29216
number of patents finished processing: 308, total: 29216
number of patents finished proc

number of patents finished processing: 440, total: 29216
number of patents finished processing: 441, total: 29216
number of patents finished processing: 442, total: 29216
number of patents finished processing: 443, total: 29216
number of patents finished processing: 444, total: 29216
number of patents finished processing: 445, total: 29216
number of patents finished processing: 446, total: 29216
number of patents finished processing: 447, total: 29216
number of patents finished processing: 448, total: 29216
number of patents finished processing: 449, total: 29216
number of patents finished processing: 450, total: 29216
number of patents finished processing: 451, total: 29216
number of patents finished processing: 452, total: 29216
number of patents finished processing: 453, total: 29216
number of patents finished processing: 454, total: 29216
number of patents finished processing: 455, total: 29216
number of patents finished processing: 456, total: 29216
number of patents finished proc

number of patents finished processing: 584, total: 29216
number of patents finished processing: 585, total: 29216
number of patents finished processing: 586, total: 29216
number of patents finished processing: 587, total: 29216
number of patents finished processing: 588, total: 29216
number of patents finished processing: 589, total: 29216
number of patents finished processing: 590, total: 29216
number of patents finished processing: 591, total: 29216
number of patents finished processing: 592, total: 29216
number of patents finished processing: 593, total: 29216
number of patents finished processing: 594, total: 29216
number of patents finished processing: 595, total: 29216
number of patents finished processing: 596, total: 29216
number of patents finished processing: 597, total: 29216
number of patents finished processing: 598, total: 29216
number of patents finished processing: 599, total: 29216
number of patents finished processing: 600, total: 29216
number of patents finished proc

...

number of patents finished processing: 28344, total: 29216
number of patents finished processing: 28345, total: 29216
number of patents finished processing: 28346, total: 29216
number of patents finished processing: 28347, total: 29216
number of patents finished processing: 28348, total: 29216
number of patents finished processing: 28349, total: 29216
number of patents finished processing: 28350, total: 29216
number of patents finished processing: 28351, total: 29216
number of patents finished processing: 28352, total: 29216
number of patents finished processing: 28353, total: 29216
number of patents finished processing: 28354, total: 29216
number of patents finished processing: 28355, total: 29216
number of patents finished processing: 28356, total: 29216
number of patents finished processing: 28357, total: 29216
number of patents finished processing: 28358, total: 29216
number of patents finished processing: 28359, total: 29216
number of patents finished processing: 28360, total: 292

number of patents finished processing: 28492, total: 29216
number of patents finished processing: 28493, total: 29216
number of patents finished processing: 28494, total: 29216
number of patents finished processing: 28495, total: 29216
number of patents finished processing: 28496, total: 29216
number of patents finished processing: 28497, total: 29216
number of patents finished processing: 28498, total: 29216
number of patents finished processing: 28499, total: 29216
number of patents finished processing: 28500, total: 29216
number of patents finished processing: 28501, total: 29216
number of patents finished processing: 28502, total: 29216
number of patents finished processing: 28503, total: 29216
number of patents finished processing: 28504, total: 29216
number of patents finished processing: 28505, total: 29216
number of patents finished processing: 28506, total: 29216
number of patents finished processing: 28507, total: 29216
number of patents finished processing: 28508, total: 292

number of patents finished processing: 28703, total: 29216
number of patents finished processing: 28704, total: 29216
number of patents finished processing: 28705, total: 29216
number of patents finished processing: 28706, total: 29216
number of patents finished processing: 28707, total: 29216
number of patents finished processing: 28708, total: 29216
number of patents finished processing: 28709, total: 29216
number of patents finished processing: 28710, total: 29216
number of patents finished processing: 28711, total: 29216
number of patents finished processing: 28712, total: 29216
number of patents finished processing: 28713, total: 29216
number of patents finished processing: 28714, total: 29216
number of patents finished processing: 28715, total: 29216
number of patents finished processing: 28716, total: 29216
number of patents finished processing: 28717, total: 29216
number of patents finished processing: 28718, total: 29216
number of patents finished processing: 28719, total: 292

number of patents finished processing: 28851, total: 29216
number of patents finished processing: 28852, total: 29216
number of patents finished processing: 28853, total: 29216
number of patents finished processing: 28854, total: 29216
number of patents finished processing: 28855, total: 29216
number of patents finished processing: 28856, total: 29216
number of patents finished processing: 28857, total: 29216
number of patents finished processing: 28858, total: 29216
number of patents finished processing: 28859, total: 29216
number of patents finished processing: 28860, total: 29216
number of patents finished processing: 28861, total: 29216
number of patents finished processing: 28862, total: 29216
number of patents finished processing: 28863, total: 29216
number of patents finished processing: 28864, total: 29216
number of patents finished processing: 28865, total: 29216
number of patents finished processing: 28866, total: 29216
number of patents finished processing: 28867, total: 292

number of patents finished processing: 29135, total: 29216
number of patents finished processing: 29136, total: 29216
number of patents finished processing: 29137, total: 29216
number of patents finished processing: 29138, total: 29216
number of patents finished processing: 29139, total: 29216
number of patents finished processing: 29140, total: 29216
number of patents finished processing: 29141, total: 29216
number of patents finished processing: 29142, total: 29216
number of patents finished processing: 29143, total: 29216
number of patents finished processing: 29144, total: 29216
number of patents finished processing: 29145, total: 29216
number of patents finished processing: 29146, total: 29216
number of patents finished processing: 29147, total: 29216
number of patents finished processing: 29148, total: 29216
number of patents finished processing: 29149, total: 29216
number of patents finished processing: 29150, total: 29216
number of patents finished processing: 29151, total: 292

In [416]:
share_internal_list = np.asarray(share_internal_list)
share_self_list = np.asarray(share_self_list)
share_external_list = np.asarray(share_external_list)
total_citation_list = np.asarray(total_citation_list)
direct_citation_list= np.asarray(direct_citation_list)

In [417]:
share_internal_stat = [len(share_internal_list),share_internal_list.mean(),
                       share_internal_list.std(),
                       np.percentile(share_internal_list,10),
                       np.percentile(share_internal_list,50),
                       np.percentile(share_internal_list,90)]

In [418]:
share_external_stat = [len(share_external_list),share_external_list.mean(),
                       share_external_list.std(),
                       np.percentile(share_external_list,10),
                       np.percentile(share_external_list,50),
                       np.percentile(share_external_list,90)]

In [419]:
share_self_stat = [len(share_self_list),share_self_list.mean(),
                       share_self_list.std(),
                       np.percentile(share_self_list,10),
                       np.percentile(share_self_list,50),
                       np.percentile(share_self_list,90)]

In [420]:
direct_citation_stat = [len(direct_citation_list),direct_citation_list.mean(),
                       direct_citation_list.std(),
                       np.percentile(direct_citation_list,10),
                       np.percentile(direct_citation_list,50),
                       np.percentile(direct_citation_list,90)]

In [421]:
total_citation_stat = [len(total_citation_list),total_citation_list.mean(),
                       total_citation_list.std(),
                       np.percentile(total_citation_list,10),
                       np.percentile(total_citation_list,50),
                       np.percentile(total_citation_list,90)]

In [422]:
table = pd.DataFrame([direct_citation_stat,
                      total_citation_stat, share_internal_stat, share_external_stat,
                      share_self_stat], columns = ['Obs','Mean','SD','10th','50th','90th'])

In [423]:
index = ['direct_citation_stat','total_citation_stat','share_internal_stat','share_external_stat',
         'share_self_stat']

In [424]:
table.index = index

In [425]:
table

,Obs,Mean,SD,10th,50th,90th
direct_citation_stat,29216,5.358297,7.935648,1.000000,3.0,11.000000
total_citation_stat,29216,669.668093,3371.600605,1.000000,26.0,985.000000
share_internal_stat,29216,0.007574,0.026775,0.000000,0.0,0.022709
share_external_stat,29216,0.915253,0.203893,0.731707,1.0,1.000000
share_self_stat,29216,0.077173,0.199073,0.000000,0.0,0.247141


In [426]:
table.to_csv('table2.csv')

# Problems:

#### 1. On page 270, Belenzon finds 201943 original patents between 1969 and 1990. I don’t know whether or not he applies the ‘Compustat firms’ constraint in this search. I got 322319 patents when not applying constraints and 156851 patents when applying constraints. In either case the figure doesn’t match.

#### 2. Actually, I got a little bit confused by the Compustat data in raw data. The assignee number cannot be directly matched with the Compustat data. I guess the CUSIP data are also provided by Compustat. So I use CUSIP data to filter out assignees that hold no record in Compustat database.

#### 3. On page 272, Belenzon says ‘I include only firms with at least one granted patent application that receives at least one citation in the period 1975-99’. I got confused by this sentence. I don’t know whether the cited patent is granted during 75-99 or the citing patent comes out during 75-99.